In [30]:
from tensorflow.keras.models import load_model

loaded_model = load_model("pitch_detection_cnn.keras")

In [36]:
import sounddevice as sd
import numpy as np
import librosa
import joblib
import soundfile as sf

notes_window = 0.3

fs = 44100
max_frames = 40
chunk_size = int(notes_window * fs)
threshold = 0.005 # 0.02

buffer = np.zeros(chunk_size)
hit_number = 0  # global counter



# Define pitch classes (C, C#, D, ..., B)
pitch_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 
               'F#', 'G', 'G#', 'A', 'A#', 'B']

ignore_samples = int(notes_window * fs)  # ignore next 0.5 sec after a hit
samples_since_last_hit = ignore_samples  # initialize above threshold

import numpy as np
pre_trigger_sec = 0.03  # 50 ms

import random

# Game state
current_target = random.choice(pitch_names)
score = 0

print(f"Play this note: {current_target}")

def process_snippet(snippet):
    global buffer  # only if you really need to use or modify it
    global current_target, score

    # Chroma CQT feature
    chroma = librosa.feature.chroma_cqt(y=snippet, sr=fs, bins_per_octave=36)
    chroma = chroma / (np.sum(chroma, axis=0, keepdims=True) + 1e-6)

    # Pad or truncate chroma to fixed length
    if chroma.shape[1] < max_frames:
        pad_width = max_frames - chroma.shape[1]
        chroma = np.pad(chroma, ((0, 0), (0, pad_width)), mode='constant')
    else:
        chroma = chroma[:, :max_frames]

    chroma_input = np.expand_dims(chroma, axis=0)  # shape: (1, 12, max_frames)

    prediction = loaded_model.predict(chroma_input)
    predicted_class = np.argmax(prediction)
    predicted_pitch = pitch_names[(predicted_class + 9) % 12]

    print("Predicted pitch:", predicted_pitch)

    # Check if it matches the target
    if predicted_pitch == current_target:
        print("✅ Correct!")
        score += 1
    else:
        print(f"❌ Incorrect! Target was {current_target}")

    print(f"Score: {score}")
    print(f"Score: {score/(hit_number+1)}")

    # Pick a new target
    current_target = random.choice(pitch_names)
    print(f"Next note: {current_target}")

    # Play snippet for verification
    #sd.play(snippet, fs)

    

ignore_samples = int(notes_window * fs)  # ignore next 0.5 sec after a hit
samples_since_last_hit = ignore_samples  # initialize above threshold

import numpy as np
pre_trigger_sec = 0.03  # 50 ms

# Check current state


def callback(indata, frames, time, status):
    global buffer, hit_number, samples_since_last_hit, was_above_threshold
    audio_chunk = indata[:, 0]  # mono
    samples_since_last_hit += len(audio_chunk)
    current_above_threshold = np.max(np.abs(audio_chunk)) > threshold
    if np.max(np.abs(audio_chunk)) > threshold and samples_since_last_hit >= ignore_samples and current_above_threshold and not was_above_threshold:
        
        print(f"Hit detected! Total hits: {hit_number}")

        # Find first sample above threshold
        above_thresh = np.where(np.abs(audio_chunk) > threshold)[0]
        if len(above_thresh) == 0:
            return
        start_idx = above_thresh[0]

        # Include pre-trigger
        pre_trigger_samples = int(pre_trigger_sec * fs)
        start_idx = max(0, start_idx - pre_trigger_samples)
        trimmed = audio_chunk[start_idx:]

        # Take exactly 0.25 s
        snippet_length = int(0.25 * fs)
        if len(trimmed) >= snippet_length:
            snippet = trimmed[:snippet_length]
        else:
            pad_width = snippet_length - len(trimmed)
            snippet = np.pad(trimmed, (0, pad_width), mode='constant')

        # Normalize amplitude
        snippet = snippet / (np.max(np.abs(snippet)) + 1e-6)


        filename = f"New Pitches 2/{pitch_names[hit_number%12]}_{hit_number//12 + 3}_real2.wav"
        #sf.write(filename, snippet, fs)

        process_snippet(snippet)
        hit_number += 1
        # Reset cooldown
        samples_since_last_hit = 0

        

    # Update circular buffer
    buffer = np.roll(buffer, -len(audio_chunk))
    buffer[-len(audio_chunk):] = audio_chunk
    was_above_threshold = current_above_threshold

with sd.InputStream(channels=1, callback=callback, samplerate=fs, blocksize=chunk_size):
    print("Recording... Press Ctrl+C to stop")
    while True:
        sd.sleep(1000)


Play this note: G
Recording... Press Ctrl+C to stop
Hit detected! Total hits: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted pitch: G
✅ Correct!
Score: 1
Score: 1.0
Next note: D#


c:\Users\TheAiro\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=690
  warnings.warn(
c:\Users\TheAiro\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=345
  warnings.warn(
c:\Users\TheAiro\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=173
  warnings.warn(
c:\Users\TheAiro\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=87
  warnings.warn(


Hit detected! Total hits: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted pitch: D#
✅ Correct!
Score: 2
Score: 1.0
Next note: F
Hit detected! Total hits: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted pitch: F
✅ Correct!
Score: 3
Score: 1.0
Next note: C#
Hit detected! Total hits: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted pitch: C#
✅ Correct!
Score: 4
Score: 1.0
Next note: A#
Hit detected! Total hits: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted pitch: A#
✅ Correct!
Score: 5
Score: 1.0
Next note: E
Hit detected! Total hits: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted pitch: E
✅ Correct!
Score: 6
Score: 1.0
Next note: C
Hit detected! Total hits: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted pitch: C
✅ Correct!
Score: 7
Score: 1.0
Next note: F#
Hit detected! Total hits: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted pitch: F#
✅ Correct!
Score: 8
Score: 1.0
Next note: F#
Hit detected! Total hits: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted pitch: F#
✅ C

KeyboardInterrupt: 